In [42]:
%reload_ext autoreload
%autoreload 2

import sys, os, random, copy, collections, time, re, argparse, commands
sys.path.append('src/')
sys.stdout = sys.stderr
import pyhocon
import numpy as np
from pprint import pprint
from logging import FileHandler
import tensorflow as tf
from src.main import Manager
from utils import common, evaluation, tf_utils
from core import models, datasets
args = common.dotDict({
    'checkpoint_path': 'checkpoints/tmp',
    'config_path': 'configs/config',
    'cleanup': True,
    'log_file': None,
    'interactive': True,
    'batch_size': 1,
})

sess = tf.InteractiveSession(graph=tf.Graph())
#sess.as_default()
manager = Manager(args, sess)
config = manager.config

Reading a config from configs/config ...
Restore the config to checkpoints/tmp/config ...
{u'num_train_data': 10000, u'dropout_rate': 0.2, u'num_layers': 1, u'char_vocab_path': u'dataset/embeddings/char_vocab.english.txt', u'decay_frequency': 100, u'w_vocab_size': 15000, u'word_max_len': 20, u'dataset_info': {u'test': {u'path': u'dataset/dailydialog/test.csv', u'max_lines': 0}, u'train': {u'path': u'dataset/dailydialog/train.csv', u'max_lines': 1000}, u'valid': {u'path': u'dataset/dailydialog/valid.csv', u'max_lines': 0}}, u'teacher_forcing': False, u'cell_type': u'GRUCell', u'c_vocab_size': 100, u'max_epoch': 30, u'wbase': True, u'decay_rate': 0.999, u'rnn_type': u'bidirectional_dynamic_rnn', u'c_embedding_size': 8, u'embeddings': [ConfigTree([(u'path', u'dataset/embeddings/glove.840B.300d.txt.filtered'), (u'size', 300), (u'format', u'txt')])], u'utterance_max_len': 25, u'learning_rate': 0.001, u'lowercase': True, u'batch_size': 1, u'dataset_type': u'DailyDialogDataset', u'max_gradien

In [43]:
batch = manager.debug()

In [48]:
b = batch.next()
w_vocab = manager.w_vocab
c_vocab = manager.c_vocab
sys.stdout = sys.stderr
#print 'w_contexts:', b.w_contexts[0]
w_contexts = [w_vocab.id2sent(x, join=True) for x in b.w_contexts[0]]
#print 'c_contexts:', b.c_contexts[0]
#print '\n'.join([c_vocab.id2sent(x, join=True) for x in b.c_contexts[0]])

print b.w_contexts[0]
print b.c_contexts[0]
print b.responses[0]
c_contexts = [c_vocab.id2sent(x, join=True) for x in b.c_contexts[0]]

#print 'response:', b.responses[0] 
responses = w_vocab.id2sent(b.responses[0], join=True)
print '==== w_context  ========'
for c in w_contexts:
    print c
print '==== c_context  ========'
for c in c_contexts:
    print c
print '==== response  ========'    
print responses


[[   6  277    7   23 5103   33  581   35   25    8    6   64  183   28
  1310   39  196    4    0    0    0    0    0    0]
 [   6  669   11  480  128   10    9 1066  192   35   27  289 4109   12
   239   58   16   80  299    4    0    0    0    0]
 [  93   11   60  199    4    6  264  396   12 1945  315   48   49   10
   289    3   35   27  100   11 7750   32   96    4]]
[array([[24,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [20, 14,  4, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [16,  6, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 7, 11,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [11,  9, 20, 12,  5, 17, 45, 14,  5,  0,  0,  0,  0,  0,  0,  0],
       [21, 12,  7,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10, 12,  7, 13, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [21,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [15,  6,  0,  0,  0,  0,  0,  0,

In [ ]:
import pandas as pd
sys.stdout = sys.stderr
nrows = 500
train_data = pd.read_csv('dataset/ubuntu-dialogue/train.csv', nrows=nrows)
valid_data = pd.read_csv('dataset/ubuntu-dialogue/valid.csv', nrows=nrows)
data = train_data
batch_size=2
utterance_max_len = 20
context_max_len = 4
_PAD = 0
_EOU = '__eou__'
_EOT = '__eot__'
_URL = '_URL'

import random
_id=random.randint(0, nrows)
from core.datasets import UbuntuDialogueDataset
#vocab = UbuntuDialogueDataset(config.dataset_info.train, config.vocab_size, config.lowercase)
vocab= UbuntuDialogueDataset.create_vocab_from_data(config.dataset_info.train, config.vocab_size, config.lowercase)
dataset = UbuntuDialogueDataset(config.dataset_info, vocab)

# if 'Utterance' in data:
#     context = [preprocess(x) for x in data['Context']]
#     response = [x for x in data['Utterance']]
#     print context[_id]
#     print response[_id]
# else:
#     context = [preprocess(x) for x in data['Context']]
#     response = [x for x in data['Ground Truth Utterance']]
#     n_distractors = 8
#     distractors = [] 
#     for i in xrange(n_distractors):
#         distractors.append([preprocess_utterance(x) for x in data['Distractor_%d' % i]])
#     distractors = list(zip(*distractors))
#     print 'ID', _id
#     print 'Context:', context[_id]
#     print 'Response:', response[_id]
#     for i, d in enumerate(distractors[_id]):
#         print 'Distractor_%d:' % i, d

In [26]:
PAD_ID=0
context_max_len=2
w_contexts = [
    [[1,2,3,4], [1,2,3,4,5]],
    [[4,5,6,7]]
]
w_contexts = [[d[i] if i < len(d) else [] for i in xrange(context_max_len)] for d in w_contexts]
utterance_max_len = 5
print [tf.keras.preprocessing.sequence.pad_sequences(
          d, maxlen=utterance_max_len, 
          padding='post', truncating='post', value=PAD_ID) for d in w_contexts]

[array([[1, 2, 3, 4, 0],
       [1, 2, 3, 4, 5]], dtype=int32), array([[4, 5, 6, 7, 0],
       [0, 0, 0, 0, 0]], dtype=int32)]
